In [23]:
!pip install ultralytics

In [25]:
#clone YOLOv10
!git clone https://github.com/THU-MIG/yolov10.git

fatal: destination path 'yolov10' already exists and is not an empty directory.


In [ ]:
!nvidia-smi #gpu check

In [ ]:
# get pretrained-model --> DONE
!wget -P /kaggle/working/yolov10 https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt

In [4]:
%cd /kaggle/working/yolov10
!pip install -r /kaggle/working/yolov10/requirements.txt

/kaggle/working/yolov10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 2.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 26.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62

## Train YOLOv10-l

In [ ]:
%cd /kaggle/working/yolov10
data_yaml_path = "/kaggle/input/tilda-farbic-yolov8/data_custom.yaml"
weight_path = "/kaggle/working/yolov10/yolov10l.pt"

!wandb disabled

!yolo task = detect mode = train epochs = 100 batch = 16 plots = True model = {weight_path} data = {data_yaml_path}

# Detect on test set

In [5]:
pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 1.6 MB/s eta 0:00:0000:01:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#create output folder
import os
output = "/kaggle/working/detections_test_YOLOv10"
os.makedirs(output)

In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLOv10

def detect(image, model, i):
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    image = cv2.imread(image_path)

    results = model(source=image, conf=0.25)[0]
    detections = sv.Detections.from_ultralytics(results)

    annotated_image = bounding_box_annotator.annotate(
        scene=image, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)

    #sv.plot_image(annotated_image)
    cv2.imwrite(os.path.join(output, 'YOLOv10_image_' + str(i) + '.png'), annotated_image)

In [ ]:
import os

model = YOLOv10('/kaggle/input/yolov10-fdd-weights/Model weight/YOLOv10_freeze.pt')
folder_path = "/kaggle/input/tilda-farbic-yolov8/test/images"
i = 0
for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    image = cv2.imread(image_path)
    detect(image, model, i)
    #print("detect image", i)
    i += 1
print("detect image completed")